In [ ]:
import xarray as xr
import hvplot.xarray
import holoviews as hv
import numpy as np
import pandas as pd

from snobedo.lib.dask_utils import start_cluster, client_ip_and_port
from snobedo.snotel import SnotelLocations, CsvParser

from common import SHARED_STORE, SNOBAL_DIR, SNOTEL_DIR, DATA_DIR, COARSEN_OPTS, HV_PLOT_OPTS

In [ ]:
client = start_cluster(6, 24)
client_ip_and_port(client)

In [ ]:
year = 2021
water_year = f'wy{year}'

## SNOTEL 

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites_2x2.json')

## Model run with HRRR clouds and solar

In [ ]:
HRRR_solar = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw_hrrr_solar/run*/net_solar.nc',
    parallel=True,
)
HRRR_cloud = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw_hrrr_solar/run*/cloud_factor.nc',
    parallel=True,
)
HRRR_therm = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw_hrrr_solar/run*/smrf_20*.nc',
    parallel=True,
)

## Model with SMRF clouds and solar 

In [ ]:
smrf_eb = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/GMD/{water_year}/erw/run*/smrf_energy*.nc',
    parallel=True,
)
smrf_out = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/GMD/{water_year}/erw/run*/smrf_20*.nc',
    parallel=True,
)

# Net Solar

In [ ]:
HV_PLOT_OPTS['ylabel'] = 'Net Solar W/m^2'

In [ ]:
butte_snobal_hrrr = HRRR_solar.net_solar.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat).coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()
schofield_snobal_hrrr = HRRR_solar.net_solar.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat).coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()
taylor_snobal_hrrr = HRRR_solar.net_solar.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat).coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()

In [ ]:
butte_snobal = smrf_eb.net_solar.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat).coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()
schofield_snobal = smrf_eb.net_solar.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat).coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()
taylor_snobal = smrf_eb.net_solar.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat).coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()

### Schofield 

In [ ]:
schofield_snobal_hrrr.hvplot(label='HRRR solar').opts(**HV_PLOT_OPTS) * schofield_snobal.hvplot(label='SMRF solar') 

### Butte

In [ ]:
butte_snobal_hrrr.hvplot(label='HRRR solar').opts(**HV_PLOT_OPTS) * butte_snobal.hvplot(label='SMRF solar') 

### Taylor 

In [ ]:
taylor_snobal_hrrr.hvplot(label='HRRR solar').opts(**HV_PLOT_OPTS) * taylor_snobal.hvplot(label='SMRF solar') 

### Rolling (daily 

In [ ]:
(schofield_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Schofield').opts(**HV_PLOT_OPTS) * \
butte_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Butte') * \
taylor_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Taylor') + \
schofield_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Schofield').opts(**HV_PLOT_OPTS) * \
butte_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Butte')  * \
taylor_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Taylor')).cols(1)

# Clouds

In [ ]:
HV_PLOT_OPTS['ylabel'] = 'Cover: 0 -> Full; 1 -> Clear'

In [ ]:
butte_snobal_hrrr = HRRR_cloud.TCDC.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat).coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()
schofield_snobal_hrrr = HRRR_cloud.TCDC.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat).coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()
taylor_snobal_hrrr = HRRR_cloud.TCDC.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat).coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()

In [ ]:
butte_snobal = smrf_out.cloud_factor.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat).compute()
butte_snobal = butte_snobal.coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()
schofield_snobal = smrf_out.cloud_factor.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat).compute()
schofield_snobal = schofield_snobal.coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()
taylor_snobal = smrf_out.cloud_factor.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat).compute()
taylor_snobal = taylor_snobal.coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()

### Schofield 

In [ ]:
schofield_snobal_hrrr.hvplot(label='HRRR clouds').opts(**HV_PLOT_OPTS) * schofield_snobal.hvplot(label='SMRF clouds') 

### Butte

In [ ]:
butte_snobal_hrrr.hvplot(label='HRRR clouds').opts(**HV_PLOT_OPTS) * butte_snobal.hvplot(label='SMRF clouds') 

### Taylor 

In [ ]:
taylor_snobal_hrrr.hvplot(label='HRRR clouds').opts(**HV_PLOT_OPTS) * taylor_snobal.hvplot(label='SMRF clouds') 

### Rolling (daily) 

In [ ]:
(schofield_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Schofield').opts(**HV_PLOT_OPTS) * \
butte_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Butte') * \
taylor_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Taylor') + \
schofield_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Schofield').opts(**HV_PLOT_OPTS) * \
butte_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Butte')  * \
taylor_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Taylor')).cols(1)

# Thermal 

In [ ]:
HV_PLOT_OPTS['ylabel'] = 'Thermal W/m^2'

In [ ]:
butte_snobal_hrrr = HRRR_therm.thermal.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat).coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()
schofield_snobal_hrrr = HRRR_therm.thermal.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat).coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()
taylor_snobal_hrrr = HRRR_therm.thermal.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat).coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()

In [ ]:
butte_snobal = smrf_out.thermal.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat).compute()
butte_snobal = butte_snobal.coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()
schofield_snobal = smrf_out.thermal.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat).compute()
schofield_snobal = schofield_snobal.coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()
taylor_snobal = smrf_out.thermal.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat).compute()
taylor_snobal = taylor_snobal.coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).compute()

### Schofield 

In [ ]:
schofield_snobal_hrrr.hvplot(label='HRRR thermal').opts(**HV_PLOT_OPTS) * schofield_snobal.hvplot(label='SMRF thermal') 

### Butte

In [ ]:
butte_snobal_hrrr.hvplot(label='HRRR thermal').opts(**HV_PLOT_OPTS) * butte_snobal.hvplot(label='SMRF thermal') 

### Taylor 

In [ ]:
taylor_snobal_hrrr.hvplot(label='HRRR thermal').opts(**HV_PLOT_OPTS) * taylor_snobal.hvplot(label='SMRF thermal') 

### Rolling (daily) 

In [ ]:
(schofield_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Schofield').opts(**HV_PLOT_OPTS) * \
butte_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Butte') * \
taylor_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Taylor') + \
schofield_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Schofield').opts(**HV_PLOT_OPTS) * \
butte_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Butte') * \
taylor_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Taylor')).cols(1)